In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [81]:
import numpy as np
from copy import deepcopy

class PathState:
    def __init__(self,loc,steps):
        self.loc = loc
        self.steps = steps

    def give_neighbors(self,garden,infinite=False ):
        neighs=[]
        nR,nC = len(garden),len(garden[0])
        
        changes = [ (-1,0), (1,0), (0,1), (0,-1) ]
        for xc in changes:
            new_loc = tuple([ i+j for i,j in zip(xc,self.loc)])
            if infinite:
                check = tuple([ new_loc[0]%nR, new_loc[1]%nC ])
            else:
                check = new_loc
        
            if (check[0] > -1 and check[1] > -1  
                and check[0] < nR and check[1] < nC):
                
                if garden[check[0]][check[1]] != '#':
                    neighs.append(PathState(new_loc, self.steps+1))
        
        return neighs
                   
    def __repr__(self):
        return f'{self.loc[0]}-{self.loc[1]}:{self.steps}'
    
    def __str__(self):
        return f'{self.loc[0]}-{self.loc[1]}'#:{self.steps}'
    


In [97]:
from collections import deque

def perform_breadth_first(garden,total=64, infinite=False):
    frontier = deque()
    start = None
    for ir, r in enumerate(garden):
        if "S" in r:
            ic = r.find('S')
            start = (ir, ic) 
            break
    
#    print(start)
    frontier.append(PathState(start,0))
    
    
    seen =[]
    count=0
    while len(frontier) > 0  :
        cur_loc = frontier.popleft()
        
        if cur_loc.steps%2 == total%2 and cur_loc.steps>0:
            count+=1
            
        if cur_loc.steps ==total:
            continue
        for n_loc in cur_loc.give_neighbors(garden, infinite=infinite):
            if str(n_loc) not in seen:
                seen.append(str(n_loc))
                frontier.append(n_loc)
                    
    return count

In [83]:
def do_part_one(file_path,total=64):
    with open(file_path,'r') as f:
        garden = [l.strip() for l in f.readlines()]
    
    nlocs = perform_breadth_first(garden,total)
    return nlocs

In [84]:
%%time
tmp = do_part_one('input_data/test_21.txt')
tmp

(5, 5)
CPU times: user 3.06 ms, sys: 2.2 ms, total: 5.25 ms
Wall time: 3.74 ms


42

In [61]:
%%time
do_part_one('input_data/day_21.txt')

(65, 65)
CPU times: user 728 ms, sys: 2.3 ms, total: 731 ms
Wall time: 733 ms


3651

# Part 2

In [161]:
def do_part_two(file_path):
    goal=26501365
    
    with open(file_path,'r') as f:
        garden = [l.strip() for l in f.readlines()]
        
    edge_size = len(garden)
    half_edge = edge_size//2
    points = [ half_edge+i*edge_size for i in range(3)]
#    points = np.arange(65,100,1)
    nlocs = [ perform_breadth_first(garden,t, infinite=True)  for t in points]
    
    
    model =np.poly1d(np.polyfit([0,1,2],nlocs,2))
    
    x = (goal - half_edge)//edge_size
    
    return model, int(model(x))

In [162]:
%%time
do_part_two('input_data/test_21.txt')
           

CPU times: user 31.9 ms, sys: 2.54 ms, total: 34.4 ms
Wall time: 34.7 ms


(poly1d([91., 25., 13.]), 528192461129798)

In [163]:
%%time
model, t = do_part_two('input_data/day_21.txt')


CPU times: user 8min 31s, sys: 118 ms, total: 8min 31s
Wall time: 8min 32s


In [164]:
t

607334325965750

In [165]:
model

poly1d([14840., 14940.,  3751.])